In [3]:
%matplotlib inline
import os
import yaml
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import numpy as np

In [4]:
base_dir = "cleaned_data"
base_dir = Path(base_dir)

In [5]:
print("Exploring dataset structure...")
print(f"Train images: {len(list((base_dir / 'lidar_data' / 'train' / 'images').glob('*.png')))}")
print(f"Validation images: {len(list((base_dir / 'lidar_data' / 'valid' / 'images').glob('*.png')))}")
print(f"Test images: {len(list((base_dir / 'lidar_data' / 'test' / 'images').glob('*.png')))}")
print(f"Training labels: {len(list((base_dir / 'lidar_data' / 'train' / 'labels').glob('*.txt')))}")

Exploring dataset structure...
Train images: 466
Validation images: 388
Test images: 192
Training labels: 1464


In [6]:
model = YOLO('yolov8n.pt')

model.train(data="/cluster/home/pettdalh/tdt4265_project/data/data.yaml", epochs=50, batch=16, imgsz=640, device="cuda")

New https://pypi.org/project/ultralytics/8.3.87 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.86 🚀 Python-3.11.11 torch-2.5.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/cluster/home/pettdalh/tdt4265_project/data/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, emb

train: Scanning /cluster/home/pettdalh/tdt4265_project/data/lidar_data/train/labels.cache... 366 images, 0 backgrounds, 3 corrupt: 100%|██████████| 369/369 [00:00<?, ?it/s]

train: WARNING ⚠️ /cluster/home/pettdalh/tdt4265_project/data/lidar_data/train/images/image_1052.png: ignoring corrupt image/label: Truncated File Read
train: WARNING ⚠️ /cluster/home/pettdalh/tdt4265_project/data/lidar_data/train/images/image_1084.png: ignoring corrupt image/label: Truncated File Read
train: WARNING ⚠️ /cluster/home/pettdalh/tdt4265_project/data/lidar_data/train/images/image_1089.png: ignoring corrupt image/label: Truncated File Read


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /cluster/home/pettdalh/tdt4265_project/data/lidar_data/valid/labels.cache... 388 images, 0 backgrounds, 2 corrupt: 100%|██████████| 390/390 [00:00<?, ?it/s]

val: WARNING ⚠️ /cluster/home/pettdalh/tdt4265_project/data/lidar_data/valid/images/image_191.png: ignoring corrupt image/label: Truncated File Read
val: WARNING ⚠️ /cluster/home/pettdalh/tdt4265_project/data/lidar_data/valid/images/image_624.png: ignoring corrupt image/label: Truncated File Read


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/23 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 76.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 31.94 MiB is free. Process 3400944 has 5.61 GiB memory in use. Process 3001985 has 2.96 GiB memory in use. Process 3003813 has 3.09 GiB memory in use. Process 3864980 has 1.07 GiB memory in use. Process 2402226 has 352.00 MiB memory in use. Process 2128276 has 966.00 MiB memory in use. Process 2457421 has 310.00 MiB memory in use. Including non-PyTorch memory, this process has 1.53 GiB memory in use. Of the allocated memory 1.16 GiB is allocated by PyTorch, and 57.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Get results without saving
results = model.predict(
    source="/cluster/home/pettdalh/tdt4265_project/data/lidar_data/test/images/image_44.png",
    device="cuda",
    conf=0.2  # Lower confidence threshold
)

NameError: name 'model' is not defined

In [ ]:
# Get result for first image
result = results[0]

# Load the original image
img = cv2.imread("/cluster/home/pettdalh/tdt4265_project/Poles/lidar/combined_color/test/image_44.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

# Create figure and axes
fig, ax = plt.subplots(1, figsize=(12, 9))
ax.imshow(img)

# Get the detection boxes
boxes = result.boxes
for box in boxes:
    # Get coordinates
    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
    confidence = float(box.conf[0])
    
    # Create rectangle
    rect = patches.Rectangle(
        (x1, y1), x2-x1, y2-y1, 
        linewidth=2, 
        edgecolor='r', 
        facecolor='none'
    )
    
    # Add rectangle to plot
    ax.add_patch(rect)
    
    # Add confidence label
    ax.text(
        x1, y1-10, 
        f"Pole: {confidence:.2f}", 
        color='white', 
        fontweight='bold',
        bbox=dict(facecolor='red', alpha=0.5)
    )

# Show plot
plt.title(f"Detected {len(boxes)} poles")
plt.axis('off')  # Hide axes
plt.tight_layout()
plt.show()

NameError: name 'results' is not defined